In [1]:
import scipy.sparse as sp
import numpy as np
import torch 
import tensorflow as tf

# Load the sparse matrix from the .npz file
A_sparse_scipy = sp.load_npz('A_sparse_scipy.npz')
p_dense_numpy = np.load('A_numpy.npy')

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
A_sparse_scipy = A_sparse_scipy.tocoo()

# Extract row, column, and data arrays from the SciPy sparse matrix
row = torch.from_numpy(A_sparse_scipy.row).long()
col = torch.from_numpy(A_sparse_scipy.col).long()
values = torch.from_numpy(A_sparse_scipy.data)

# Stack row and column indices to create a 2 x N tensor of indices
indices = torch.stack([row, col])

# Create the PyTorch sparse tensor
A_torch = torch.sparse_coo_tensor(indices, values, size=A_sparse_scipy.shape, dtype=torch.float32, device="cuda:0")
p_torch = torch.tensor(p_dense_numpy, dtype=torch.float32, device="cuda:0")

In [ ]:
%timeit torch.sparse.mm(A_torch, p_torch)

3.51 ms ± 2.37 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
# Extract row indices, column indices, and data values
indices = np.vstack((A_sparse_scipy.row, A_sparse_scipy.col)).T
values = A_sparse_scipy.data
dense_shape = A_sparse_scipy.shape

# Create a TensorFlow SparseTensor on GPU
with tf.device('/GPU:0'):  # Specify the GPU device
    A_tf = tf.SparseTensor(indices=tf.convert_to_tensor(indices, dtype=tf.int64),
                                   values=tf.convert_to_tensor(values, dtype=tf.float32),
                                   dense_shape=tf.convert_to_tensor(dense_shape, dtype=tf.int64))
    p_tf = tf.convert_to_tensor(p_dense_numpy, dtype=tf.float32)

I0000 00:00:1730740916.427316  253471 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22216 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:08:00.0, compute capability: 8.6
I0000 00:00:1730740916.427767  253471 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22430 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:41:00.0, compute capability: 8.6


In [ ]:
%timeit tf.sparse.sparse_dense_matmul(A_tf, tf.reshape(p_tf, (-1, 1)))

275 µs ± 9.15 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
